In [2]:
import cv2

# Load the image
image = cv2.imread('mrunal1.png')

# Define new dimensions
new_width = 300
new_height = 200
new_dimensions = (new_width, new_height)

# Resize the image
resized_image = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_LINEAR)

# Save or display the resized image
cv2.imwrite('resized_image.jpg', resized_image)
cv2.imshow('Original Image', image)
cv2.imshow('Resized Image', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2

# Load the image
image = cv2.imread('flowers.jpg')

# Define scale factors
scale_x = 0.5  # Scale down to 50% of the original width
scale_y = 0.5  # Scale down to 50% of the original height

# Resize the image
resized_image = cv2.resize(image, None, fx=scale_x, fy=scale_y, interpolation=cv2.INTER_LINEAR)

# Save or display the resized image
cv2.imwrite('resized_image_scaled.jpg', resized_image)
cv2.imshow('Resized Image', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
# Maintaining Aspect Ratio
import cv2

# Load the image
image = cv2.imread('sai1.png')

# Get original dimensions
original_height, original_width = image.shape[:2]

# Desired width
desired_width = 400

# Calculate the aspect ratio
aspect_ratio = original_height / original_width

# Compute new dimensions
new_width = desired_width
new_height = int(desired_width * aspect_ratio)

# Resize the image
resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

# Save or display the resized image
# cv2.imwrite('resized_image_aspect_ratio.jpg', resized_image)
cv2.imshow('Resized Image with Aspect Ratio', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


To rotate an image using OpenCV, it's essential to understand the underlying affine transformation process, which combines rotation and translation operations. Here's a structured breakdown:

### Core Rotation Matrix

In a 2D Cartesian coordinate system, rotating a point $(x, y)$ around the origin by an angle $\theta$ counterclockwise is achieved using the rotation matrix:

$$
R = \begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{bmatrix}
$$

This matrix, when multiplied with the coordinate vector of a point, yields the new coordinates after rotation.

### Adjusting for Arbitrary Rotation Centers

Images are typically rotated around their center rather than the origin. To rotate an image around a specific center point $(c_x, c_y)$, the transformation involves:

1. **Translation to the Origin**: Shift the image so that the center of rotation aligns with the origin.
2. **Rotation**: Apply the rotation matrix $R$.
3. **Translation Back to Original Position**: Shift the image back to its original position.

Combining these steps, the transformation matrix $M$ becomes:

$$
M = \begin{bmatrix}
\cos\theta & -\sin\theta & (1 - \cos\theta) \cdot c_x + \sin\theta \cdot c_y \\
\sin\theta & \cos\theta & -\sin\theta \cdot c_x + (1 - \cos\theta) \cdot c_y
\end{bmatrix}
$$

This matrix accounts for both rotation and the necessary translations to rotate around the desired center.

### Bounding Box Calculation to Prevent Cropping

Rotating an image can cause parts of it to be cropped if the output image dimensions are not adjusted. To ensure the entire rotated image fits within the frame, the new dimensions can be calculated as:

$$
\text{new\_width} = h \cdot |\sin\theta| + w \cdot |\cos\theta|
$$

$$
\text{new\_height} = h \cdot |\cos\theta| + w \cdot |\sin\theta|
$$

Where $w$ and $h$ are the original width and height of the image, respectively. This calculation ensures that the rotated image is fully contained within the new dimensions.

### OpenCV Implementation

OpenCV provides the `cv2.getRotationMatrix2D()` function, which simplifies the creation of the rotation matrix. The function's syntax is:

$$
M = \text{cv2.getRotationMatrix2D}((c_x, c_y), \theta, \text{scale})
$$

Where:

- $(c_x, c_y)$: Center of rotation.
- $\theta$: Rotation angle in degrees.
- `scale`: Scaling factor.

The resulting matrix $M$ is then used with `cv2.warpAffine()` to apply the rotation:

$$
\text{rotated\_image} = \text{cv2.warpAffine}(\text{image}, M, (\text{new\_width}, \text{new\_height}))
$$

This function applies the affine transformation defined by $M$ to the image, producing the rotated output.

Understanding these mathematical foundations allows for precise control over image rotation, ensuring that transformations are performed as intended without unintended cropping or distortion. 

In [14]:
import cv2
import numpy as np

# Load image
image = cv2.imread("image.png")  # Change path as needed
(h, w) = image.shape[:2]  # Get height and width

angle = 30  # Rotation angle
center = (w // 2, h // 2)  # Find center of image

# Get the rotation matrix
M = cv2.getRotationMatrix2D(center, angle, 1.0)

# Compute new bounding box size to prevent cropping
cos_theta = np.abs(M[0, 0])  # cos(theta)
sin_theta = np.abs(M[0, 1])  # sin(theta)

new_w = int((h * sin_theta) + (w * cos_theta))
new_h = int((h * cos_theta) + (w * sin_theta))

# Adjust the rotation matrix to consider translation
M[0, 2] += (new_w / 2) - center[0]
M[1, 2] += (new_h / 2) - center[1]

# Apply rotation
rotated = cv2.warpAffine(image, M, (new_w, new_h))

# Resize rotated image to match original height
rotated_resized = cv2.resize(rotated, (w, h))  # Resize to original width and height

# Show result
hstacked = np.hstack([image, rotated_resized])
cv2.imshow("Rotated Image", hstacked)
cv2.waitKey(0)
cv2.destroyAllWindows()


### **Blur Image**

### Image Blurring Techniques

Image blurring is a fundamental image processing technique used for various purposes, including noise reduction, detail smoothing, and creating special effects. Here are three common blurring methods:

**1. Gaussian Blurring:**

* **Definition:** Gaussian blur is achieved by convolving an image with a Gaussian function (a bell-shaped curve).
* **Purpose:**
    * Reduces image noise.
    * Smooths out details.
    * Used as a preprocessing step in machine learning and deep learning models.
* **Mechanism:** Each pixel's value is replaced by a weighted average of its neighboring pixels, where the weights are determined by a Gaussian distribution.

**2. Median Blurring:**

* **Definition:** Median blur uses a median filter, a non-linear digital filtering technique.
* **Purpose:**
    * Removes noise, especially "salt and pepper" noise (random black and white pixels).
    * Preserves edges better than Gaussian blur under certain conditions.
* **Mechanism:** Each pixel's value is replaced by the median value of its neighboring pixels.

**3. Bilateral Blurring:**

* **Definition:** Bilateral filtering is a non-linear, edge-preserving, and noise-reducing smoothing filter.
* **Purpose:**
    * Reduces noise while preserving sharp edges.
    * Smooths details without blurring prominent features.
* **Mechanism:**
    * Replaces each pixel's intensity with a weighted average of its neighboring pixels.
    * The weights consider both the spatial distance between pixels and the difference in their intensity values.
    * Weights can be based on a Gaussian distribution.
    * This ensures that pixels with similar intensities have a stronger influence on each other, preserving edges.


**Key Considerations:**

* **Kernel Size:** The size of the kernel (the neighborhood of pixels used for averaging) affects the degree of blurring. Larger kernels result in more significant blurring.
* **Noise Type:** Different blurring methods are more effective for specific types of noise. Median blur is excellent for salt and pepper noise, while Gaussian blur is better for Gaussian noise.
* **Edge Preservation:** Bilateral blur is designed to preserve edges, making it suitable for applications where detail retention is crucial.

**Applications:**

* **Image Editing:** Smoothing skin, reducing blemishes, creating depth of field effects.
* **Computer Vision:** Preprocessing images for feature extraction, noise reduction before object detection.
* **Medical Imaging:** Noise reduction in medical scans.
* **Astronomy:** Reducing atmospheric noise in astronomical images.


In [1]:
import cv2
import numpy as np

image = cv2.imread("mrunal1.png")  
gaussian_blur = cv2.GaussianBlur(image, (5, 5), 0)

cv2.imshow("Gaussian Blurred Image", gaussian_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
image = cv2.imread("image.png")  
median_blur = cv2.medianBlur(image, 5)

cv2.imshow("Median Blurred Image", median_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
image = cv2.imread("image.png")  
bilateral_blur = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

cv2.imshow("Bilateral Blurred Image", bilateral_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np

# Load the original image
image = cv2.imread("girl.jpg")  # Change this to your actual image path

# Ensure the image was loaded successfully
if image is None:
    raise ValueError("Image not found. Please check the file path.")

# Apply different blurring techniques
average_blur = cv2.blur(image, (5, 5))  # Averaging Blur
gaussian_blur = cv2.GaussianBlur(image, (5, 5), 0)  # Gaussian Blur
median_blur = cv2.medianBlur(image, 5)  # Median Blur

# Resize the original image to make it smaller if it's too large
max_height = 300
aspect_ratio = image.shape[1] / image.shape[0]
new_width = int(max_height * aspect_ratio)

# Resize all images to the same dimensions
image = cv2.resize(image, (new_width, max_height))
average_blur = cv2.resize(average_blur, (new_width, max_height))
gaussian_blur = cv2.resize(gaussian_blur, (new_width, max_height))
median_blur = cv2.resize(median_blur, (new_width, max_height))

# Create labels for each image
def add_label(img, label):
    h, w = img.shape[:2]
    result = img.copy()
    cv2.putText(result, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    return result

image = add_label(image, "Original")
average_blur = add_label(average_blur, "Average Blur")
gaussian_blur = add_label(gaussian_blur, "Gaussian Blur")
median_blur = add_label(median_blur, "Median Blur")

# Create a 2x2 grid layout
top_row = np.hstack([image, average_blur])
bottom_row = np.hstack([gaussian_blur, median_blur])
grid_display = np.vstack([top_row, bottom_row])

# Show the combined image
cv2.imshow("Blurring Techniques - 2x2 Grid", grid_display)
cv2.waitKey(0)
cv2.destroyAllWindows()

### **Save image**

In [5]:
import cv2
import numpy as np

In [10]:
img = cv2.imread('girl.jpg')
re_img = cv2.resize(img, (300, 450))

h = np.hstack((re_img, re_img))

cv2.imwrite("new_girl.png",h)

cv2.imshow("Img", h)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### MakeBorder using OpenCV

In [17]:
import cv2
import numpy as np

# Read image
img = cv2.imread('boy.jpg')

# Define border size
border_size = 50  # Big border (Increase for a bolder effect)

# Add a thick red border
img1 = cv2.copyMakeBorder(src=img, 
                          top=border_size, 
                          bottom=border_size, 
                          left=border_size, 
                          right=border_size, 
                          borderType=cv2.BORDER_CONSTANT,
                          value=(0, 0, 255))  # Red border (BGR format)

# Resize both images to match dimensions (keep aspect ratio)
height = 500  # Fixed height
width = 300   # Fixed width
img_resized = cv2.resize(img, (width, height))
img1_resized = cv2.resize(img1, (width, height))

# Stack side by side
hstack_img = np.hstack((img_resized, img1_resized))

# Show the stacked images
cv2.imshow("Original vs Bold Red Bordered", hstack_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [18]:
import cv2
import numpy as np

# Read image
img = cv2.imread('boy.jpg')

# Define different border sizes
thin_border = 10   # Small border (default)
medium_border = 50  # Medium bold border
thick_border = 100  # Very thick border

# Apply borders
img_thin = cv2.copyMakeBorder(img, thin_border, thin_border, thin_border, thin_border, cv2.BORDER_CONSTANT, value=(255, 255, 255))  # White border
img_medium = cv2.copyMakeBorder(img, medium_border, medium_border, medium_border, medium_border, cv2.BORDER_CONSTANT, value=(0, 0, 255))  # Medium Red border
img_thick = cv2.copyMakeBorder(img, thick_border, thick_border, thick_border, thick_border, cv2.BORDER_CONSTANT, value=(0, 0, 255))  # Thick Red border

# Resize all images to be the same size (for stacking)
width, height = 300, 500
img_resized = cv2.resize(img, (width, height))
img_thin_resized = cv2.resize(img_thin, (width, height))
img_medium_resized = cv2.resize(img_medium, (width, height))
img_thick_resized = cv2.resize(img_thick, (width, height))

# Stack all images side by side
hstack_img = np.hstack((img_resized, img_thin_resized, img_medium_resized, img_thick_resized))

# Show the stacked images
cv2.imshow("Original | Thin Border | Medium Red Border | Thick Red Border", hstack_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Read image
img = cv2.imread('flower.jpg')

# Define different border sizes
thin_border = 10
medium_border = 50
thick_border = 100

# Apply different border types
img_reflect = cv2.copyMakeBorder(img, thin_border, thin_border, thin_border, thin_border, cv2.BORDER_REFLECT)  # Reflect border
img_wrap = cv2.copyMakeBorder(img, medium_border, medium_border, medium_border, medium_border, cv2.BORDER_WRAP)  # Wrap border
img_replicate = cv2.copyMakeBorder(img, thick_border, thick_border, thick_border, thick_border, cv2.BORDER_REPLICATE)  # Replicate border
img_dark = cv2.copyMakeBorder(img, thick_border, thick_border, thick_border, thick_border, cv2.BORDER_CONSTANT, value=(0, 0, 0))  # Dark black border

# Resize all images to the same size for proper stacking
width, height = 300, 500
img_resized = cv2.resize(img, (width, height))
img_reflect_resized = cv2.resize(img_reflect, (width, height))
img_wrap_resized = cv2.resize(img_wrap, (width, height))
img_replicate_resized = cv2.resize(img_replicate, (width, height))
img_dark_resized = cv2.resize(img_dark, (width, height))

# Stack all images side by side
hstack_img = np.hstack((img_resized, img_reflect_resized, img_wrap_resized, img_replicate_resized, img_dark_resized))

# Show the stacked images
cv2.imshow("Original | Reflect | Wrap | Replicate | Dark Border", hstack_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
